In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
api_token = os.getenv("AIPROXY_TOKEN")
import requests
import pandas as pd
import subprocess
import sys
import shlex
from dateutil.parser import parse



In [48]:
headers={
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
}
url="https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"

In [85]:
def make_relative_path(path: str) -> str:
    print("I have seen you kind time and time again")
    return path.lstrip("/")  # If absolute local path, remove leading slash


Function Inits

In [50]:
#generate code

def execute_generated_code(code, arguments=None):
    # Save the provided code to a temporary file.
    script_path = "generated_script.py"
    with open(script_path, "w") as f:
        f.write(code)

    # Build command with optional arguments.
    if arguments:
        # Convert arguments to command-line format: --key value
        param_str = " ".join(f'--{k} {v}' for k, v in arguments.items())
        command = f"python {script_path} {param_str}"
    else:
        command = f"python {script_path}"

    # Execute the script and capture output.
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout if result.stdout else result.stderr


In [51]:
#run npx package

def run_npx_package(package, arg=None):
    npx = "npx.cmd" if os.name == "nt" else "npx"
    cmd = [npx, package]
    if arg:
        cmd.extend(shlex.split(arg))
    print("Running command:", " ".join(cmd))
    result = subprocess.run(cmd)
    if result.returncode:
        sys.exit(result.returncode)


In [ ]:
#count_day_occurrences


def count_day_occurrences(input_path, output_path, day):
    input_path = make_relative_path(input_path)
    output_path = make_relative_path(output_path)
    
    day = day.lower()
    day_map = {
        "monday": 0,
        "tuesday": 1,
        "wednesday": 2,
        "thursday": 3,
        "friday": 4,
        "saturday": 5,
        "sunday": 6
    }
    

    target_weekday = day_map[day]
    count = 0

    with open(input_path, 'r') as infile:
        for line in infile:
            line = line.strip()
            if not line:
                continue
            try:
                dt = parse(line, fuzzy=True)
                iso_date = dt.strftime('%Y-%m-%d')
                if dt.weekday() == target_weekday:
                    count += 1
            except Exception:
                continue

    with open(output_path, 'w') as outfile:
        outfile.write(str(count))

# count_day_occurrences('/data/dates.txt', '/data/dates-wednesdays.txt', 'Wednesday')


In [76]:
#A4 sort some json files

def sort_json_file(input_file, output_file, sort_keys):
    input_file = make_relative_path(input_file)
    output_file = make_relative_path(output_file)
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    sorted_data = sorted(data, key=lambda x: [x[k] for k in sort_keys])
    
    with open(output_file, 'w') as f:
        json.dump(sorted_data, f, indent=4)

In [77]:
sort_json_file('/data/contacts.json', '/data/contacts-sorted.json', ['last_name', 'first_name'])

OUT HERE
OUT HERE


Function Calling Schema

In [78]:
function_list = [ {
        "type": "function",
        "function": {
            "name": "execute_generated_code",
            "description": "Execute a given Python script with provided parameters.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The Python code to execute."
                    },
                    "arguments": {
                        "type": "object",
                        "description": "Key-value pairs of arguments to be passed to the script.",
                        "additionalProperties": True  
                    }
                },
                "required": ["code"], 
                "additionalProperties": False 
            }
        }
    },
{
        "type": "function",
        "function": {
            "name": "count_day_occurrence",
            "description": "Count occurrences of a specified day from dates in a file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "input_file": {"type": "string", "description": "Input file with dates."},
                    "day": {"type": "string", "description": "Day to count."},
                    "output_file": {"type": "string", "description": "Output file for count."}
                },
                "required": ["input_file", "day", "output_file"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "run_npx_package",
            "description": "Run npx command with optional arg and output file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "package": {"type": "string"},
                    "arg": {"type": "string"},
                    "output_file": {"type": "string"}
                },
                "required": ["package"],
                "additionalProperties": False
            }
        }
    },
    {
  "type": "function",
  "function": {
    "name": "sort_json_file",
    "description": "Sort JSON file by keys.",
    "parameters": {
      "type": "object",
      "properties": {
        "input_file": { "type": "string" },
        "output_file": { "type": "string" },
        "sort_keys": {
          "type": "array",
          "items": { "type": "string" }
        }
      },
      "required": ["input_file", "output_file", "sort_keys"],
      "additionalProperties": False
    }
  }
}



]

In [79]:
prompt="""
sort the json file at location /data/contacts.json by email and save it as /data/contacts-email-sorted.json
"""

In [80]:
response = requests.post(
    url=url,
    headers=headers,
    json={
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": "You are an expert Python programmer."},
            {"role": "user", "content": prompt}
        ],
        "tools": function_list,
        "tool_choice": "auto"
    }
)

In [81]:
response.json()

{'id': 'chatcmpl-B0GqeL2rTMAvbaMd6tMMqbzTjWN0h',
 'object': 'chat.completion',
 'created': 1739404464,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_a1LayPQIdwPUE9hXXJZaw9Vb',
      'type': 'function',
      'function': {'name': 'sort_json_file',
       'arguments': '{"input_file":"/data/contacts.json","output_file":"/data/contacts-email-sorted.json","sort_keys":["email"]}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 210,
  'completion_tokens': 39,
  'total_tokens': 249,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_bd83329f63',
 'monthlyCost': 0.034461000000000006,
 'cost': 0.000864,
 'monthl

In [82]:
import json
arguments_str = response.json()['choices'][0]['message']['tool_calls'][0]['function']['arguments']
parsed_arguments = json.loads(arguments_str)

print(parsed_arguments)


{'input_file': '/data/contacts.json', 'output_file': '/data/contacts-email-sorted.json', 'sort_keys': ['email']}


In [83]:
function_name=response.json()['choices'][0]['message']['tool_calls'][0]['function']['name']


In [67]:
if function_name == "execute_generated_code":
    code=parsed_arguments['code']
    arguments = parsed_arguments.get('arguments', None)
    execute_generated_code(code, arguments)

In [68]:
if function_name=="run_npx_package":
    package=parsed_arguments['package']
    arg=parsed_arguments.get('arg', None)
    run_npx_package(package, arg)

In [69]:
if function_name=="count_day_occurrence":
    input_file=parsed_arguments['input_file']
    output_file=parsed_arguments['output_file']
    day=parsed_arguments['day']
    count_day_occurrences(input_file, output_file, day)

OUT HERE
OUT HERE


In [86]:
if function_name=="sort_json_file":
    input_file=parsed_arguments['input_file']
    output_file=parsed_arguments['output_file']
    sort_keys=parsed_arguments['sort_keys']
    sort_json_file(input_file, output_file, sort_keys)

I have seen you kind time and time again
I have seen you kind time and time again
